In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai.imports import *
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *
PATH = "data/seedlings/"
sz=224

In [3]:
arch=resnet34
tfms = tfms_from_model(resnet34, sz, aug_tfms=transforms_top_down, max_zoom=1.1)
data = ImageClassifierData.from_paths(PATH, tfms=tfms_from_model(arch, sz), test_name='test')
arch=resnet34
learn = ConvLearner.pretrained(arch, data, precompute=False)

In [5]:
learn.load('311_dropout_unfrozen')

In [6]:
learn.predict(is_test=True)

array([[ -1.98249,  -8.51929,  -9.82385, ...,  -9.02756,  -8.86165,  -7.56024],
       [-12.72483,  -5.38926,  -2.7539 , ...,  -3.86631,  -0.15059, -10.72198],
       [-14.73188, -11.47322,  -8.79628, ...,  -2.70292,  -0.0848 , -13.37899],
       ..., 
       [ -3.19007,  -5.04943,  -5.27114, ...,  -7.09248,  -5.46881,  -3.04132],
       [-13.20412, -11.05331,  -4.99197, ...,  -6.22431, -10.48567,  -8.0016 ],
       [ -6.91658,  -8.90037,  -2.57651, ...,  -5.77794,  -3.46446,  -3.62459]], dtype=float32)

In [7]:
log_preds, y = learn.TTA(is_test=True, n_aug=4)
preds = np.mean(log_preds, 0)
probs = np.mean(np.exp(log_preds), axis=0)
indices = probs.argmax(axis=1)

In [8]:
classes = [data.classes[i] for i in indices]

In [9]:
import csv
filenames = os.listdir(f'{PATH}test')

In [10]:
with open(f'{PATH}submission.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(('file', 'species'))
    writer.writerows(zip(filenames, classes))
    for x in range (0, 234):
        writer.writerow('none', 'none')